# Types of errors

In [ ]:
# Run this cell to play back an audio file; type Esc-o to hyde the player
from IPython.display import Audio
Audio("media/exc-intro.mp3")

There are two types of errors: 
* **syntax errors**: the compiler spots them, you fix them
* **runtime errors**: can be handled via the *exception* mechanism
* **logical errors** (aka **bugs**): here your program may not even crash, but it misbehaves (eg messes up your data) because of a logical flaw in the code. If I knew how to catch these I would be a rich man, so don't let's talk about them any longer. As I said, two types.

Syntax errors are the most common type of error you encounter when you start programming, but are easily spotted and fixed (the interpreter helps you a lot):

In [ ]:
print "Hello world

spam=1
ham=3

Runtime errors are much trickier, as they are conditions that depend on program execution and/or on the data the user may enter:

In [ ]:
gg=input ("Number of guesses? ")
rr=input ("Number of right answers? ")
guesses=float(gg)
right=float(rr)
ratio=guesses/right
print ("You took " + str(ratio) + " guesses for each correct answer")

In the above example, the code is correct and the program normally works; however, errors may arise because of the data (in itself acceptable or otherwise) supplied by the user. These errors trigger *Exceptions*, which (if unhandled) lead to program termination. Notice here that the name of the built-in exception (such as *ZeroDivisonError* or *ValueError*) describes what has happened.

Typical among operations that can cause exceptions are input/output operations (especially involving files). One can try to open a file that does not exist, write to a directory without writing permissions, or encounter unexpected input while reading the file. 

In [ ]:
# run this cell to show a video, use slider to resize it, type Esc-o to hide it
from IPython.display import Video, clear_output; from ipywidgets import interactive, IntSlider
def _play(resize): display(Video(filename="media/exc-whatswrong.webm",data="",width=resize))
interactive(_play, resize=IntSlider(min=150, max=900, step=50, value=600, continuous_update=False, readout=False))

# Exception handling

An exception need not be the end of the program: it can be handled with the following construct:

```
try:
    # anything that can cause errors
    BLOCK
except <EXCEPTION> [as e]: # type of exception and "as" clause are optional
    BLOCK
```

Here ```<EXCEPTION>``` stands for the type of exception we want to handle. Technically it is a class, so that the actual exception is (you guessed it) an object of that class; it gets assigned to whatever variable is specified in the optional ```as``` clause, so that you can inspect it in the following ```BLOCK```. This will become clearer with the examples below.

For instance, we can modify the previous code as follows:

In [ ]:
try:
    gg=input ("Number of guesses? ")
    rr=input ("Number of right answers? ")
    guesses=float(gg)
    right=float(rr)
    ratio=guesses/right
    print ("You took " + str(ratio) + " guesses for each correct answer")
except:
    print ("You loser!")

In [ ]:
# run this cell to show a video, use slider to resize it, type Esc-o to hide it
from IPython.display import Video, clear_output; from ipywidgets import interactive, IntSlider
def _play(resize): display(Video(filename="media/exc-tryexcept.webm",data="",width=resize))
interactive(_play, resize=IntSlider(min=150, max=900, step=50, value=600, continuous_update=False, readout=False))

As we see, the problem here is that all errors trigger the "You loser" response, while for instance the program should ask for input again if the user did not enter numbers. It is possible to use more than one ```except``` clause, each restricted to handling only one type of errors:

In [ ]:
while True:
    try:
        gg=input ("Number of guesses? ")
        rr=input ("Number of right answers? ")
        guesses=float(gg)
        right=float(rr)
        ratio=guesses/right
        print("You took " + str(ratio) + " guesses for each correct answer")
        break # we are done
    except ZeroDivisionError: # handles division by zero
        print("You loser!")
        break # exit the loop 
    except ValueError: # handles invalid input
        print("Please enter numbers")


restricting ```except``` clauses in this way is actually a good idea, because a "wildcard" ```except``` can mask a true programming error.

In [ ]:
# run this cell to show a video, use slider to resize it, type Esc-o to hide it
from IPython.display import Video, clear_output; from ipywidgets import interactive, IntSlider
def _play(resize): display(Video(filename="media/exc-dangers.webm",data="",width=resize))
interactive(_play, resize=IntSlider(min=150, max=900, step=50, value=600, continuous_update=False, readout=False))

As said exceptions are actually objects, and carry some information about the error that has occurred. This can be shown to the user by printing the exception, or handled programmatically via the ```args``` attribute of the exception object:

In [ ]:
while True:
    try:
        gg=input ("Number of guesses? ")
        rr=input ("Number of right answers? ")
        guesses=float(gg)
        right=float(rr)
        ratio=guesses/right
        print ("You took " + str(ratio) + " guesses for each correct answer")
        break # we are done
    except ZeroDivisionError: # handles division by zero
        print ("You loser!")
        break
    except ValueError as e: # stores the exception object in variable e
        print ("This error has occurred:")
        print (e) # prints the contents of e.args


# Exception propagation

In general, an exception will happen in some function or method that has been called by some other function or method, which in turn may have been called by something else. An exception will propagate up the call stack until it is handled. This means that an exception happening in a function can be handled by the caller. If nothing handles it, the exception causes the program to terminate. Consider the following examples:

In [ ]:
def average(l):
    return sum(l)/ len(l)


print (average([3, 4, 5]))
print (average([]))

As we can see, an exception happens in function ```average```, called by the main program, if the list passed to ```average``` is empty. This exception can be handled inside the function as follows:

In [ ]:
def average(l):
    try:
        avg= sum(l)/ len(l)
        return avg
    except ZeroDivisionError:
        return None # a predefined placeholder for no value


print (average([3, 4, 5]))
print (average([]))

or it can be handled by the caller:

In [ ]:
def average(l):
    return sum(l)/ len(l)

try:
    print (average([3, 4, 5]))
    print (average([]))
except ZeroDivisionError:
    print ("eeek!")

the exception still happens in the function, but it is propagated back to the caller and handled there.

In [ ]:
# run this cell to show a video, use slider to resize it, type Esc-o to hide it
from IPython.display import Video, clear_output; from ipywidgets import interactive, IntSlider
def _play(resize): display(Video(filename="media/exc-propagation.webm",data="",width=resize))
interactive(_play, resize=IntSlider(min=150, max=900, step=50, value=600, continuous_update=False, readout=False))

# Rolling your own

Especially when programming a library, it may be useful to declare your own exceptions. You can do that by extending the ```Exception``` class, as shown below. In a Bioinformatics library, for instance, a ```SeqError``` may be defined to flag errors occurring while dealing with a DNA sequence (in this case, a search for a character that is not a nucleotide). When the condition occurs, your code can use the keyword ```raise``` to throw a ```SeqError``` than then behaves like any other exception, and can be handled in a ```try``` - ```except``` clause.

In [ ]:
class SeqError(Exception): # inherits from Exception
    """ A user-defined exception """
    def __init__(self, message, value):
        super().__init__() # call constructor of superclass
        self.message=message
        self.value=value
        
    def __str__(self):
        return f"Value {self.value} caused an error: {self.message}"
    

class DNASequence:
    def __init__(self, seq):
        self.seq=seq
        
    def countNucleotide(self,n):
        """ returns the number of times a nucleotide n appears in the sequence;
        raises a SeqError if n is not a nucleotide """
        if n not in "ACTG":
            # This triggers the exception
            raise SeqError("Not a Nucleotide", n)
        return self.seq.count(n)

myseq=DNASequence("AATCGATG")
# try adding a try - except SeqError: clause around these instructions
myseq.countNucleotide('A')
myseq.countNucleotide('Q')        

Note that the ```Exception``` class provides some boilerplate code, so that if you simply define ```SeqError``` as follows:
```
class SeqError(Exception):
    pass
```
you get almost the same behaviour (don't take my word for it, try it!). The point here is to show you that these are *bona-fide* objects and as such they are entirely customisable.

In fact, exception handling can even make use of the inheritance relations among exception classes, as described in the video below.

In [ ]:
# run this cell to show a video, use slider to resize it, type Esc-o to hide it
from IPython.display import Video, clear_output; from ipywidgets import interactive, IntSlider
def _play(resize): display(Video(filename="media/exc-inheritance.webm",data="",width=resize))
interactive(_play, resize=IntSlider(min=150, max=900, step=50, value=600, continuous_update=False, readout=False))

All [built-in exceptions](https://docs.python.org/3/library/exceptions.html) are listed in the Python documentation; each of them can obviously be extended, should you ever wish to tweak them.

**(C) 2014,2020 Fabrizio Smeraldi** ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)), all rights reserved. In: "Coding for Scientists", School of Biological and Chemical Sciences, Queen Mary University of London.